In [15]:
import codecs
import itertools

In [16]:
import sys
import nltk
from nltk.stem.porter import *
from sklearn.feature_extraction import stop_words
import xml.etree.cElementTree as ET
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import zipfile
import os
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
# from wordcloud import WordCloud
PARTIALS = False
from nltk.tokenize import RegexpTokenizer

In [17]:
file = open("expanded_glove_20_words.txt", "r+")
uncertainty_str = file.read()
import ast 
dictionary_orig_glove_file = ast.literal_eval(uncertainty_str)

In [18]:
def get_keys_and_values(dictionary):
    key_list=list(dictionary.keys())
    value_list=list(itertools.chain.from_iterable(dictionary.values()))
    word_list=list(set(key_list+value_list))
    return word_list

In [19]:
expanded_glove_dict=get_keys_and_values(dictionary_orig_glove_file)

In [20]:
len(expanded_glove_dict)

3667

In [21]:
path = '/ifs/gsb/usf_interns/Parser_Project/ParsedDocumentsFolder/10KParsed/Regression_10Ks/'
files=os.listdir(path)

In [22]:
files[2550]

'parsed_10-K_0001193125-14-078778.txt'

In [23]:
corpus=dict()
for file in files:
    text = codecs.open(path+file, 'r', encoding='utf-8', errors='ignore').read()
    corpus.update({file:text})

In [24]:
def tokenizer(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    tokenizer = RegexpTokenizer(r'\w+')
    valid_characters = string.printable
    text = ''.join(i for i in text if i in valid_characters)
    text = text.lower()
    text = re.sub('[' + string.punctuation + '0-9\\r\\t\\n\®]', ' ', text)
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if len(w) > 2]  # ignore a, an, to, at, be, ...
    goodWords = [w for w in tokens if w not in ENGLISH_STOP_WORDS]
    return(goodWords)

In [25]:
def compute_tfidf(corpus):
    """
    Create and return a TfidfVectorizer object after training it on
    the list of articles pulled from the corpus dictionary. Meaning,
    call fit() on the list of document strings, which figures out
    all the inverse document frequencies (IDF) for use later by
    the transform() function. The corpus argument is a dictionary 
    mapping file name to xml text.
    """

    tfidf = TfidfVectorizer(input='content',
                        analyzer='word',
                        tokenizer=tokenizer,
                        stop_words='english',
                        decode_error = 'ignore')
    list_of_articles=[]
    for key in corpus:
        list_of_articles.append(corpus[key])      
    tfidf_matrix=tfidf.fit(list_of_articles)  
    return tfidf_matrix 

In [26]:
tfidf=compute_tfidf(corpus)

In [27]:
def summarize(tfidf, text):
    """
    Given a trained TfidfVectorizer object and some XML text, return
    up to n (word,score) pairs in a list. Discard any terms with
    scores < 0.09.
    """
    tmatrix=tfidf.transform([text])
    retTuple=[]
    feature_index=tmatrix.nonzero()[1]
    feature_words=tfidf.get_feature_names()
    for i in feature_index:
        retTuple.append((feature_words[i],tmatrix[0,i]))

    def Sort_Tuple(tup):  
        return(sorted(tup, key = lambda x: x[1],reverse = True))
    
    sorted_list=Sort_Tuple(retTuple)
#     sortedlistn=(sorted_list[:n])
#     sortedlistnFinal=[i for i in sortedlistn if i[1] >= 0.09]
    return(sorted_list)

In [28]:
outputTfidfEachDict={}
outputTfidfTotalDict_glove={}
for key in files:
    Text_10K=corpus[key]
    tfidf_scores=summarize(tfidf, Text_10K)
    dict_10={}
    for item in tfidf_scores:
        dict_10.update( {item[0] :item[1]} )
    uncertainty_dict={}
    total_tfidf_unc=0
    for word in expanded_glove_dict:
        if word in dict_10:
            uncertainty_dict[word]=dict_10[word]
            total_tfidf_unc=total_tfidf_unc+dict_10[word] 
    outputTfidfEachDict[key] = uncertainty_dict
    outputTfidfTotalDict_glove[key]= total_tfidf_unc
            

In [29]:
with open('saved_uncertainty_tfidf_dicts.txt', 'w') as f:
    f.write(str(outputTfidfEachDict))

In [30]:
%store outputTfidfTotalDict_glove

Stored 'outputTfidfTotalDict_glove' (dict)


In [31]:
outputTfidfTotalDict_glove

{'parsed_10-K_0000882835-13-000008.txt': 3.0723713813860876,
 'parsed_10-K_0001193125-10-053838.txt': 3.4140115885107427,
 'parsed_10-K_0000320187-13-000092.txt': 3.1543184363957426,
 'parsed_10-K_0001095073-14-000008.txt': 2.5827091541204856,
 'parsed_10-K_0000004904-13-000025.txt': 1.7564534438709394,
 'parsed_10-K_0001308179-13-000024.txt': 1.7182252276339098,
 'parsed_10-K_0000950123-11-018684.txt': 2.831272301903062,
 'parsed_10-K_0001193125-12-078626.txt': 3.5154932174962252,
 'parsed_10-K_0000874761-15-000009.txt': 3.0744340577118794,
 'parsed_10-K_0001521332-14-000011.txt': 2.3900659213407653,
 'parsed_10-K_0001090872-13-000029.txt': 3.472913511657516,
 'parsed_10-K_0001358071-14-000005.txt': 2.918802354778253,
 'parsed_10-K_0000078239-11-000015.txt': 1.0007623241983556,
 'parsed_10-K_0000021344-14-000008.txt': 2.48355034077235,
 'parsed_10-K_0000898173-13-000006.txt': 3.116749198807614,
 'parsed_10-K_0001047469-12-001182.txt': 2.8654071827965995,
 'parsed_10-K_0001193125-11-04

Code for Single file:

In [ ]:
key='parsed_10-K_0000006201-13-000023.txt'
Text_10K=corpus[key]
tfidf_scores=summarize(tfidf, Text_10K)
dict_10={}
for item in tfidf_scores:
    dict_10.update( {item[0] :item[1]} )
    

In [ ]:
uncertainty_dict={}
total_tfidf_unc=0
for word in uncertainty_list:
    if word in dict_10:
        uncertainty_dict[word]=dict_10[word]
        total_tfidf_unc=total_tfidf_unc+dict_10[word]

In [ ]:
total_tfidf_unc

In [ ]:
uncertainty_dict